# Task 2: Exploratory Data Analysis
## Ethiopia Financial Inclusion Forecast

This notebook performs comprehensive exploratory data analysis to understand patterns and factors influencing financial inclusion in Ethiopia.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except OSError:
    plt.style.use('seaborn-darkgrid')
sns.set_palette("husl")

# Create output directory for figures
Path("../reports/figures").mkdir(parents=True, exist_ok=True)

# Load data with error handling
try:
    data_path = Path("../data/raw/ethiopia_fi_unified_data - ethiopia_fi_unified_data.csv")
    if not data_path.exists():
        raise FileNotFoundError(f"Data file not found: {data_path}")
    df = pd.read_csv(data_path)
    print(f"✓ Dataset loaded successfully: {len(df)} records")
    print(f"✓ Columns: {len(df.columns)}")
except FileNotFoundError as e:
    print(f"✗ Error: {e}")
    raise
except Exception as e:
    print(f"✗ Error loading data: {e}")
    raise

# Prepare observations dataframe
obs_df = df[df['record_type'] == 'observation'].copy()
obs_df['observation_date'] = pd.to_datetime(obs_df['observation_date'], errors='coerce')
obs_df = obs_df.dropna(subset=['observation_date']).sort_values('observation_date')

print(f"✓ Observations prepared: {len(obs_df)} records with valid dates")

Dataset loaded: 72 records
Columns: 35


## 1. Dataset Overview

This section provides a comprehensive overview of the dataset structure, data quality, and temporal coverage.

In [ ]:
# 1.1 Record Type Distribution
print("=" * 80)
print("1. DATASET OVERVIEW")
print("=" * 80)

record_type_counts = df['record_type'].value_counts()
print("\nRecord Type Distribution:")
print(record_type_counts)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].pie(record_type_counts.values, labels=record_type_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Distribution by Record Type', fontweight='bold')
record_type_counts.plot(kind='bar', ax=axes[1], color='steelblue')
axes[1].set_title('Record Type Counts', fontweight='bold')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('../reports/figures/01_record_type_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Insight: Dataset contains 34 observations, 23 impact links, 12 events, and 3 targets.")
print("   The impact links connect events to indicators, enabling causal analysis.")

In [ ]:
# 1.2 Pillar Distribution
obs_pillars = obs_df['pillar'].value_counts()
print("\nPillar Distribution (Observations):")
print(obs_pillars)

fig, ax = plt.subplots(figsize=(10, 6))
obs_pillars.plot(kind='barh', ax=ax, color='coral')
ax.set_title('Observation Counts by Pillar', fontweight='bold')
ax.set_xlabel('Count')
plt.tight_layout()
plt.savefig('../reports/figures/02_pillar_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Insight: ACCESS (15) and USAGE (12) pillars dominate the observations.")
print("   GENDER (4), AFFORDABILITY (1), QUALITY (1), and TRUST (1) have limited coverage.")
print("   This indicates data gaps in quality and trust metrics.")

In [ ]:
# 1.3 Data Quality - Confidence Levels
confidence_counts = df['confidence'].value_counts()
print("\nData Quality - Confidence Levels:")
print(confidence_counts)

fig, ax = plt.subplots(figsize=(8, 6))
colors = {'high': 'green', 'medium': 'orange', 'low': 'red', 'estimated': 'gray'}
color_list = [colors.get(level, 'blue') for level in confidence_counts.index]
confidence_counts.plot(kind='bar', ax=ax, color=color_list)
ax.set_title('Data Quality: Confidence Level Distribution', fontweight='bold')
ax.set_ylabel('Count')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('../reports/figures/04_confidence_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Insight: 56 records (77.8%) have high confidence - strong data quality.")
print("   14 records (19.4%) are medium confidence, 2 (2.8%) are low confidence.")
print("   Most data comes from verified sources (operators, regulators, surveys).")

## 2. Access Analysis

Analyzing account ownership trends, growth rates, and gender gaps to understand access patterns.

In [ ]:
# 2.1 Account Ownership Trajectory
print("\n" + "=" * 80)
print("2. ACCESS ANALYSIS")
print("=" * 80)

acc_ownership = obs_df[
    (obs_df['indicator_code'] == 'ACC_OWNERSHIP') &
    (obs_df['gender'] == 'all') &
    (obs_df['location'] == 'national')
].copy().sort_values('observation_date')

if len(acc_ownership) == 0:
    print("⚠️ Warning: No account ownership data found")
else:
    print("\nAccount Ownership Timeline:")
    for _, row in acc_ownership.iterrows():
        print(f"  {row['observation_date'].strftime('%Y-%m-%d')}: {row['value_numeric']:.1f}%")
    
    # Plot trajectory
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(acc_ownership['observation_date'], acc_ownership['value_numeric'], 
            marker='o', linewidth=2, markersize=10, color='steelblue', label='Account Ownership')
    ax.set_title('Ethiopia Account Ownership Trajectory (2014-2024)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Year')
    ax.set_ylabel('Account Ownership Rate (%)')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Add growth rate annotations
    for i in range(1, len(acc_ownership)):
        prev_val = acc_ownership.iloc[i-1]['value_numeric']
        curr_val = acc_ownership.iloc[i]['value_numeric']
        growth = curr_val - prev_val
        years_diff = (acc_ownership.iloc[i]['observation_date'] - acc_ownership.iloc[i-1]['observation_date']).days / 365.25
        annual_growth = growth / years_diff
        
        mid_date = acc_ownership.iloc[i-1]['observation_date'] + (acc_ownership.iloc[i]['observation_date'] - acc_ownership.iloc[i-1]['observation_date']) / 2
        ax.annotate(f'+{growth:.1f}pp\n({annual_growth:.1f}pp/yr)', 
                    xy=(mid_date, (prev_val + curr_val) / 2),
                    xytext=(10, 10), textcoords='offset points',
                    fontsize=9, bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    plt.tight_layout()
    plt.savefig('../reports/figures/06_account_ownership_trajectory.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n📊 Insight: Account ownership grew from 22% (2014) to 49% (2024).")
    print("   Growth rate slowed significantly: +13pp (2014-2017), +11pp (2017-2021), +3pp (2021-2024).")
    print("   This slowdown occurred despite massive mobile money expansion (65M+ accounts).")

In [ ]:
# 2.2 Gender Gap Analysis
acc_male = obs_df[
    (obs_df['indicator_code'] == 'ACC_OWNERSHIP') &
    (obs_df['gender'] == 'male') &
    (obs_df['location'] == 'national')
].copy().sort_values('observation_date')

acc_female = obs_df[
    (obs_df['indicator_code'] == 'ACC_OWNERSHIP') &
    (obs_df['gender'] == 'female') &
    (obs_df['location'] == 'national')
].copy().sort_values('observation_date')

gap_data = obs_df[
    (obs_df['indicator_code'] == 'GEN_GAP_ACC') &
    (obs_df['location'] == 'national')
].copy().sort_values('observation_date')

if len(acc_male) > 0 and len(acc_female) > 0:
    gender_comparison = pd.merge(
        acc_male[['observation_date', 'value_numeric']],
        acc_female[['observation_date', 'value_numeric']],
        on='observation_date',
        suffixes=('_male', '_female')
    )
    
    print("\nGender-Disaggregated Account Ownership:")
    for _, row in gender_comparison.iterrows():
        gap = row['value_numeric_male'] - row['value_numeric_female']
        print(f"  {row['observation_date'].strftime('%Y')}: Male {row['value_numeric_male']:.1f}% vs Female {row['value_numeric_female']:.1f}% (Gap: {gap:.1f}pp)")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    axes[0].plot(acc_male['observation_date'], acc_male['value_numeric'], 
               marker='o', linewidth=2, markersize=8, label='Male', color='steelblue')
    axes[0].plot(acc_female['observation_date'], acc_female['value_numeric'], 
               marker='s', linewidth=2, markersize=8, label='Female', color='coral')
    axes[0].set_title('Account Ownership by Gender', fontweight='bold')
    axes[0].set_xlabel('Year')
    axes[0].set_ylabel('Account Ownership Rate (%)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    if len(gap_data) > 0:
        axes[1].plot(gap_data['observation_date'], gap_data['value_numeric'], 
                    marker='o', linewidth=2, markersize=8, color='red')
        axes[1].set_title('Gender Gap (Male - Female, percentage points)', fontweight='bold')
        axes[1].set_xlabel('Year')
        axes[1].set_ylabel('Gap (pp)')
        axes[1].grid(True, alpha=0.3)
        axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.savefig('../reports/figures/08_gender_gap_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n📊 Insight: Gender gap persists at 18-20pp despite overall growth.")
    print("   In 2021: 56% male vs 36% female (20pp gap).")
    print("   Women hold only 14% of mobile money accounts - far from parity.")
    print("   Growth is not gender-neutral; targeted interventions needed.")
else:
    print("⚠️ Limited gender-disaggregated data available")

## 3. Usage (Digital Payments) Analysis

Examining mobile money penetration, P2P transactions, and the registered vs active user gap.

In [ ]:
# 3.1 Mobile Money Account Penetration
print("\n" + "=" * 80)
print("3. USAGE (DIGITAL PAYMENTS) ANALYSIS")
print("=" * 80)

mm_accounts = obs_df[
    (obs_df['indicator_code'] == 'ACC_MM_ACCOUNT') &
    (obs_df['gender'] == 'all') &
    (obs_df['location'] == 'national')
].copy().sort_values('observation_date')

if len(mm_accounts) > 0:
    print("\nMobile Money Account Penetration:")
    for _, row in mm_accounts.iterrows():
        print(f"  {row['observation_date'].strftime('%Y')}: {row['value_numeric']:.2f}%")
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(mm_accounts['observation_date'], mm_accounts['value_numeric'], 
           marker='o', linewidth=2, markersize=10, color='green', label='Mobile Money Account Rate')
    ax.set_title('Mobile Money Account Penetration (2014-2024)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Year')
    ax.set_ylabel('Mobile Money Account Rate (%)')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    if len(mm_accounts) >= 2:
        for i in range(1, len(mm_accounts)):
            prev_val = mm_accounts.iloc[i-1]['value_numeric']
            curr_val = mm_accounts.iloc[i]['value_numeric']
            growth = curr_val - prev_val
            pct_growth = ((curr_val / prev_val) - 1) * 100
            
            mid_date = mm_accounts.iloc[i-1]['observation_date'] + (mm_accounts.iloc[i]['observation_date'] - mm_accounts.iloc[i-1]['observation_date']) / 2
            ax.annotate(f'+{growth:.2f}pp\n({pct_growth:.0f}% growth)', 
                       xy=(mid_date, (prev_val + curr_val) / 2),
                       xytext=(10, 10), textcoords='offset points',
                       fontsize=9, bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.7))
    
    plt.tight_layout()
    plt.savefig('../reports/figures/10_mobile_money_penetration.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n📊 Insight: Mobile money accounts doubled from 4.7% (2021) to 9.45% (2024).")
    print("   M-Pesa launch (Aug 2023) added 10.8M users, driving significant growth.")
    print("   Competition between Telebirr and M-Pesa expanded market access.")
else:
    print("⚠️ No mobile money account data found")

In [ ]:
# 3.2 P2P Transactions and Registered vs Active Gap
p2p_count = obs_df[
    (obs_df['indicator_code'] == 'USG_P2P_COUNT') &
    (obs_df['location'] == 'national')
].copy().sort_values('observation_date')

mpesa_users = obs_df[
    (obs_df['indicator_code'] == 'USG_MPESA_USERS') &
    (obs_df['location'] == 'national')
].copy().sort_values('observation_date')

mpesa_active = obs_df[
    (obs_df['indicator_code'] == 'USG_MPESA_ACTIVE') &
    (obs_df['location'] == 'national')
].copy().sort_values('observation_date')

if len(p2p_count) > 0:
    print("\nP2P Transaction Growth:")
    for _, row in p2p_count.iterrows():
        print(f"  {row['observation_date'].strftime('%Y')}: {row['value_numeric']:,.0f} transactions")
    
    if len(p2p_count) >= 2:
        prev = p2p_count.iloc[-2]['value_numeric']
        curr = p2p_count.iloc[-1]['value_numeric']
        growth_pct = ((curr / prev) - 1) * 100
        print(f"  YoY Growth: {growth_pct:.1f}%")

if len(mpesa_users) > 0 and len(mpesa_active) > 0:
    mpesa_merged = pd.merge(
        mpesa_users[['observation_date', 'value_numeric']],
        mpesa_active[['observation_date', 'value_numeric']],
        on='observation_date',
        suffixes=('_registered', '_active')
    )
    
    if len(mpesa_merged) > 0:
        mpesa_merged['activity_rate'] = (mpesa_merged['value_numeric_active'] / mpesa_merged['value_numeric_registered']) * 100
        
        print("\nRegistered vs Active Users (M-Pesa):")
        for _, row in mpesa_merged.iterrows():
            print(f"  {row['observation_date'].strftime('%Y')}:")
            print(f"    Registered: {row['value_numeric_registered']:,.0f}M")
            print(f"    90-day Active: {row['value_numeric_active']:,.0f}M")
            print(f"    Activity Rate: {row['activity_rate']:.1f}%")
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        x = np.arange(len(mpesa_merged))
        width = 0.35
        axes[0].bar(x - width/2, mpesa_merged['value_numeric_registered'], width, label='Registered', color='steelblue')
        axes[0].bar(x + width/2, mpesa_merged['value_numeric_active'], width, label='90-day Active', color='coral')
        axes[0].set_ylabel('Users (Millions)')
        axes[0].set_title('M-Pesa: Registered vs Active Users', fontweight='bold')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels([d.strftime('%Y') for d in mpesa_merged['observation_date']])
        axes[0].legend()
        axes[0].grid(True, alpha=0.3, axis='y')
        
        axes[1].bar(x, mpesa_merged['activity_rate'], color='green', alpha=0.7)
        axes[1].set_ylabel('Activity Rate (%)')
        axes[1].set_title('M-Pesa Activity Rate', fontweight='bold')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels([d.strftime('%Y') for d in mpesa_merged['observation_date']])
        axes[1].axhline(y=50, color='red', linestyle='--', alpha=0.5, label='50% benchmark')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig('../reports/figures/11_registered_vs_active.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("\n📊 Insight: M-Pesa has 66% activity rate (7.1M active / 10.8M registered).")
        print("   This registered vs active gap may explain why account ownership growth slowed.")
        print("   Many mobile money accounts may be inactive, not captured in Findex surveys.")

## 4. Event Timeline and Impact Analysis

Visualizing events overlaid on indicator trends to identify relationships and impact patterns.

In [ ]:
# 4.1 Event Timeline
print("\n" + "=" * 80)
print("4. EVENT TIMELINE AND IMPACT ANALYSIS")
print("=" * 80)

events_df = df[df['record_type'] == 'event'].copy()
events_df['observation_date'] = pd.to_datetime(events_df['observation_date'], errors='coerce')
events_df = events_df.dropna(subset=['observation_date']).sort_values('observation_date')

print("\nAll Cataloged Events:")
for _, event in events_df.iterrows():
    print(f"  {event['observation_date'].strftime('%Y-%m-%d')}: {event['indicator']} ({event['category']})")

# Timeline visualization
category_colors = {
    'product_launch': 'green',
    'market_entry': 'blue',
    'policy': 'purple',
    'regulation': 'orange',
    'infrastructure': 'red',
    'partnership': 'cyan',
    'milestone': 'gold',
    'economic': 'brown',
    'pricing': 'pink'
}

fig, ax = plt.subplots(figsize=(14, 8))

y_positions = {}
y_counter = 0
for _, event in events_df.iterrows():
    category = event['category']
    date = event['observation_date']
    
    if category not in y_positions:
        y_positions[category] = y_counter
        y_counter += 1
    
    y_pos = y_positions[category]
    color = category_colors.get(category, 'gray')
    
    ax.scatter(date, y_pos, s=200, c=color, alpha=0.7, edgecolors='black', linewidth=1)
    ax.annotate(event['indicator'][:30], 
               xy=(date, y_pos),
               xytext=(10, 0), textcoords='offset points',
               fontsize=8, rotation=0, ha='left', va='center')

ax.set_yticks(list(y_positions.values()))
ax.set_yticklabels(list(y_positions.keys()))
ax.set_xlabel('Date')
ax.set_title('Event Timeline: Financial Inclusion Events in Ethiopia', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../reports/figures/14_event_timeline.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Insight: 12 events cataloged from 2021-2025.")
print("   Product launches (Telebirr, M-Pesa) and infrastructure (Fayda, 4G) dominate.")
print("   Policy events (NFIS-II, KYC updates) create enabling conditions.")

In [ ]:
# 4.2 Events Overlaid on Indicator Trends
if len(acc_ownership) > 0 and len(mm_accounts) > 0:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Account Ownership with events
    axes[0].plot(acc_ownership['observation_date'], acc_ownership['value_numeric'], 
                marker='o', linewidth=2, markersize=8, color='steelblue', label='Account Ownership', zorder=2)
    
    key_events = ['Telebirr Launch', 'M-Pesa Ethiopia Launch', 'Safaricom Ethiopia Commercial Launch', 
                  'Fayda Digital ID Program Rollout', 'NFIS-II Strategy Launch']
    for _, event in events_df.iterrows():
        if any(key in event['indicator'] for key in key_events):
            axes[0].axvline(x=event['observation_date'], color='red', linestyle='--', alpha=0.5, linewidth=1)
            axes[0].annotate(event['indicator'][:25], 
                           xy=(event['observation_date'], axes[0].get_ylim()[1] * 0.95),
                           rotation=90, ha='right', va='top', fontsize=7)
    
    axes[0].set_title('Account Ownership with Key Events', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Account Ownership (%)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Mobile Money Accounts with events
    axes[1].plot(mm_accounts['observation_date'], mm_accounts['value_numeric'], 
                marker='s', linewidth=2, markersize=8, color='green', label='Mobile Money Accounts', zorder=2)
    
    mm_events = ['Telebirr Launch', 'M-Pesa Ethiopia Launch']
    for _, event in events_df.iterrows():
        if any(key in event['indicator'] for key in mm_events):
            axes[1].axvline(x=event['observation_date'], color='red', linestyle='--', alpha=0.5, linewidth=1)
            axes[1].annotate(event['indicator'][:25], 
                           xy=(event['observation_date'], axes[1].get_ylim()[1] * 0.95),
                           rotation=90, ha='right', va='top', fontsize=7)
    
    axes[1].set_title('Mobile Money Account Penetration with Key Events', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Year')
    axes[1].set_ylabel('Mobile Money Account Rate (%)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../reports/figures/15_events_overlaid_trends.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n📊 Insight: Telebirr launch (May 2021) preceded mobile money growth.")
    print("   M-Pesa launch (Aug 2023) drove P2P transaction surge (158% YoY).")
    print("   Safaricom market entry (Aug 2022) broke monopoly, enabled competition.")
    print("   Event timing shows clear impact patterns with 6-24 month lags.")

## 5. Key Insights Summary

Based on the analysis above, here are the key insights with supporting evidence from the visualizations.

### Insight 1: Account Ownership Growth Slowed Despite Massive Mobile Money Expansion

**Evidence (see Figure 06_account_ownership_trajectory.png):**
- Account ownership grew only **+3pp (46% → 49%)** from 2021-2024
- Previous period (2017-2021) saw **+11.0pp growth** over 4 years = **+2.75pp/year**
- Slowdown period (2021-2024) saw **+3pp over 3 years = +1.0pp/year** (63% slower)
- Meanwhile, **65M+ mobile money accounts** were opened

**Implication:** This paradox suggests mobile money accounts may not be fully captured in Findex surveys or many accounts are inactive.

### Insight 2: Mobile Money Penetration Doubled After M-Pesa Entry

**Evidence (see Figure 10_mobile_money_penetration.png):**
- Mobile money accounts: **4.7% (2021) → 9.45% (2024)** = **+101% growth**
- **M-Pesa added 10.8M registered users** by end of 2024
- **P2P transactions grew 158% YoY** after M-Pesa launch

**Implication:** Competition drives adoption. Second entrant significantly expanded market.

### Insight 3: Gender Gap Persists Despite Overall Growth

**Evidence (see Figure 08_gender_gap_analysis.png):**
- Gender gap: **20pp in 2021** (56% male vs 36% female)
- Estimated gap: **18pp in 2024** (slight improvement but still large)
- **Women hold only 14% of mobile money accounts**

**Implication:** Growth is not gender-neutral. Targeted interventions needed.

### Insight 4: Event Timing Shows Clear Impact Patterns

**Evidence (see Figure 15_events_overlaid_trends.png):**
- **Telebirr launch (May 2021)** preceded mobile money account growth
- **M-Pesa launch (Aug 2023)** drove P2P transaction surge
- **Safaricom market entry (Aug 2022)** broke monopoly, enabled competition

**Implication:** Product launches and market competition are key drivers. Impact modeling should capture event-driven dynamics.

### Insight 5: Registered vs Active Gap May Explain Slowdown

**Evidence (see Figure 11_registered_vs_active.png):**
- M-Pesa: **10.8M registered** but only **7.1M 90-day active** (66% activity rate)
- If similar rates apply to Telebirr, many accounts may be dormant

**Implication:** Need to model "registered vs active" gap separately. Inactive accounts may not be captured in surveys.

---

For complete analysis and all visualizations, see `eda_insights_summary.md` and `reports/figures/`.